In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import praw
import os
import seaborn as sns
import re

In [2]:
from dotenv import load_dotenv
load_dotenv()

user_agent = "Trump defo 1.0 by /u/speedylean"
reddit = praw.Reddit(
    client_id = os.getenv('REDDIT_ID'),
    client_secret = os.getenv('REDDIT_SECRET'),
    user_agent = "not a bot"
)

#### Trump scraper

In [ ]:
import time
import logging
import os
import praw
import pandas as pd
import re
from prawcore.exceptions import TooManyRequests, RequestException
from dotenv import load_dotenv

# filtros trump
trump_keywords = [
    'donald trump', 'trump', 'donald', 'donaldtrump', 
    'orange', 'duck', 'donaldduck', 'donald duck', 
    'donnybaby', 'donnyboy', 'donnybrook', 'trumpinator', 
    'trumpamaniac', 'trumpastrophie', 'trumpocalypse', 
    'trumpenstein', 'trumpletoes', 'tricky trump', 
    'pumpkin', 'corn', 'president trump', 
    'drumpf', 'the donald', 'mr. trump', 'potus'
    ]
trump_keywords = [keyword.lower() for keyword in trump_keywords]

# filtros trumpers
trumpers_keywords = [
    'mike pence', 'pence', 'mike', 'mikepence',
    'ron desantis', 'desantis', 'ron', 'rondesantis',
    'marjorie taylor greene', 'mtg', 'greene', 'marjorie',
    'lauren boebert', 'boebert', 'lauren',
    'ted cruz', 'cruz', 'ted', 'tedcruz',
    'mitch mcconnell', 'mcconnell', 'mitch', 'mcconnell',
    'lindsey graham', 'graham', 'lindsey', 'lindseygraham',
    'kevin mccarthy', 'mccarthy', 'kevin', 'kevinmccarthy',
    'rudy giuliani', 'giuliani', 'rudy', 'rudygiuliani',
    'steve bannon', 'bannon', 'steve', 'stevebannon',
    'michael flynn', 'flynn', 'michael', 'michaelflynn',
    'roger stone', 'roger', 'stone', 'rogerstone',
    'matt gaetz', 'gaetz', 'matt', 'mattgaetz',
    'sean hannity', 'hannity', 'sean', 'seanhannity'
    ]
trumpers_keywords = [keyword.lower() for keyword in trumpers_keywords]

# filtros kamala
kamala_keywords = [
    'kamala harris', 'kamala', 'harris', 'kamalaharris', 
    'vice president harris', 'vp harris', 'kammie', 
    'kammy', 'kamalalal', 'kamalita', 'mrs. harris', 'ms. harris'
    'Comrade Kamala', 'Crazy Kamala', 'Laffin Kamala', 'Lying Kamala Harris', 'Kamabla'
    ]
kamala_keywords = [keyword.lower() for keyword in kamala_keywords]

# filtros kamalers
kamalers_keywords = [

    'nancy pelosi', 'pelosi', 'nancy', 'nancypelosi',
    'chuck schumer', 'schumer', 'chuck', 'chuckschumer',
    'elizabeth warren', 'warren', 'elizabeth', 'elizabethwarren',
    'bernie sanders', 'sanders', 'bernie', 'berniesanders',
    'aoc', 'alexandria ocasio-cortez', 'ocasio-cortez', 'alexandria',
    'pete buttigieg', 'buttigieg', 'pete', 'petebuttigieg',
    'gavin newsom', 'newsom', 'gavin', 'gavinnewsom',
    'cory booker', 'booker', 'cory', 'corybooker',
    'stacey abrams', 'abrams', 'stacey', 'staceyabrams',
    'hillary', 'michelle obama', 'michelle', 'michelleobama',
    'keisha lance bottoms', 'keisha', 'lance bottoms', 'keisha lance'
]
kamalers_keywords = [keyword.lower() for keyword in kamalers_keywords]

democrats_keywords = [
    'democrat', 'democrats', 'dem party', 'democratic party',
    'dnc', 'liberal', 'liberals', 'left', 'leftist', 
    'progressive', 'progressives', 'blue', 'blue wave', 
    'blue states', 'donkey', 'donkeys', 'libtard', 
    'libs', 'snowflake', 'snowflakes', 'blue team',
    'the left', 'dem establishment', 'biden administration', 
    'left-wing', 'democrat-controlled', 'blue collar',
    'progressive caucus', 'leftists', 'dem agenda'
]
democrats_keywords = [keyword.lower() for keyword in democrats_keywords]

republicans_keywords = [
    'republican', 'republicans', 'gop', 'grand old party',
    'conservative', 'conservatives', 'right', 'right-wing',
    'red', 'red wave', 'red states', 'elephant', 
    'elephants', 'right-winger', 'right-wingers', 
    'maga', 'make america great again', 'trump party', 
    'gop establishment', 'rnc', 'red team', 
    'the right', 'republican-controlled', 'republican agenda'
]
republicans_keywords = [keyword.lower() for keyword in republicans_keywords]

# haciendo logging para monitorear errores de rate-limit y retries
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers = [logging.StreamHandler(), logging.FileHandler(r"C:\Users\34616\Documents\4GEEKS\datos_gordos\reddit\Scraping_results\logs\all_hotscraper_v4.log", mode= 'a')])


# Inicializando Reddit
load_dotenv()
user_agent = "political_research 3.0 by /u/speedylean"
reddit = praw.Reddit(
    client_id = os.getenv('REDDIT_ID'),
    client_secret = os.getenv('REDDIT_SECRET'),
    user_agent = user_agent
)

# funcion para checkear si tiene keyword
def contains_trump_keyword(text):
    return any(keyword in text.lower() for keyword in trump_keywords)
def contains_trumper_keyword(text):
    return any(keyword in text.lower() for keyword in trumpers_keywords)
def contains_kamala_keyword(text):
    return any(keyword in text.lower() for keyword in kamala_keywords)
def contains_kamaler_keyword(text):
    return any(keyword in text.lower() for keyword in kamalers_keywords)
def contains_democrat_keyword(text):
    return any(keyword in text.lower() for keyword in democrats_keywords)
def contains_republican_keyword(text):
    return any(keyword in text.lower() for keyword in republicans_keywords)


def is_relevant_content(text):
    text = text.lower()
    # frases comunes de bots:
    bot_phrases = [
        "i am a bot", "this bot", "automoderator", 
        "bot created", "beep boop", "bleep bloop", 
        "bot detected", "this is a reminder from the bots",
        "your post has been removed", "your comment has been removed", 
        "moderator action", "subreddit rules", 
        "thank you for your submission", "follow the subreddit rules", 
        "please read our community guidelines", "crosspost", "x-post"
    ]

    # filtering comments that contain these phrases
    if any(phrase in text for phrase in bot_phrases):
        return False
    
    # filter out urls
    if re.search(r'http[s]?://', text):
        return False
    
    # filtering out short generic comments
    if len(text) < 20 or text in ['thanks', 'lol', 'ok', 'i agree']:
        return False

    return True


# definiendo la lista de subreddits:
subreddits = ['SocialDemocracy', 'USPolitics', 'Libertarian', 'ConservativesOnly', 'OurPresident', 'Socialism', 'WayOfTheBern', 'politics', 'PoliticalDiscussion', 'Conservative', 'Liberal', 'ModeratePolitics', 'Ask_Politics', 'Democrats', 'Republican']

# columnas del dataframe
columns = ['text', 'submission_type', 'subreddit', 'labels', 'id']

# Iniciando variables de backoff:
initial_backoff = 5
max_backoff = 300
backoff_factor = 2

output_file = r"C:\Users\34616\Documents\4GEEKS\datos_gordos\reddit\Scraping_results\all_hotscrape_v4.csv"

# funcion para guardar progreso y liberar memoria
def save_progress(df):
    global i
    if not df.empty: # guardar solo si NO está vacío
        try:
            logging.info(f"Saving current progress...")
            # checkeamos si el df existe antes de guardarlo:
            file_exists = os.path.isfile(output_file)

            # si existe guardamos SIN headers:
            df.to_csv(output_file, mode='a', header=not file_exists, index=False)
            logging.info(f"Saved {len(df)} records successfully. i = {i}")

            # no limpiamos el buffer aquí?

        except Exception as e:
            logging.error(f"Error saving progress: {e}")
            logging.info(f"df.shape = {df.shape}, i = {i}")

def backoff_sleep(attempt):
    sleep_time = min(initial_backoff * (backoff_factor ** attempt), max_backoff)
    logging.info(f"Rate limit hit. Sleeping for {sleep_time} seconds...")
    time.sleep(sleep_time)

# funcion scrapeadora:
# daily avg posts in subreddit = 400 (one week = 2400)
limit = None # automatic limit is 1000

def all_hotscraped():
    global initial_backoff, i
    i = 0
    records_collected = 0
    n_unique_posts = 0
    save_threshold = 50000
    df_buffer = pd.DataFrame(columns=columns)

    # loopeando y filtrando subreddits, posts, comments y replies
    for subreddit in subreddits:
        subreddit_name = subreddit
        subreddit = reddit.subreddit(subreddit)
        logging.info(f"Now scraping r/{subreddit_name}")
        id_list = []

        # scrappear los posts dentro de un while loop para manejar los errores y reintentos
        attempt = 0
        while True:
            try:
                posts = list(subreddit.hot(limit=limit))
                logging.info(f"Fetched {len(posts)} posts from r/{subreddit_name}")
                n_unique_posts += len(posts)
                break  # en caso de éxito salimos del loop
            except TooManyRequests as e:
                logging.warning(f"Rate limit hit when fetching posts from r/{subreddit_name}: {e}")
                backoff_sleep(attempt)
                attempt += 1
            except RequestException as e:
                logging.error(f"Request exception when fetching posts from r/{subreddit_name}: {e}")
                backoff_sleep(attempt)
                attempt += 1
            except Exception as e:
                logging.error(f"Unexpected error when fetching posts from r/{subreddit_name}: {e}")
                break  # Saltar al siguiente subreddit
        

        for post_idx, post in enumerate(posts):
            id = post.id
            if id not in id_list:
                id_list.append(id)

                 # filtrar y guardar el título            
                title_lables = []
                if contains_trump_keyword(post.title):
                    title_lables.append('trump')
                if contains_trumper_keyword(post.title):
                    title_lables.append('trumper')
                if contains_kamala_keyword(post.title):
                    title_lables.append('kamala')
                if contains_kamaler_keyword(post.title):
                    title_lables.append('kamaler')
                if contains_democrat_keyword(post.title):
                    title_lables.append('democrat')
                if contains_republican_keyword(post.title):
                    title_lables.append('republicans')
                if len(title_lables) == 0:
                    title_lables.append('none')

                new_row = pd.DataFrame([{
                    'text': post.title,
                    'submission_type': 'title',
                    'subreddit': subreddit_name,
                    'labels': title_lables,
                    'id': id
                }])
                df_buffer = pd.concat([df_buffer, new_row], ignore_index=True)
                records_collected += 1
                i +=1
            
                # while loop para los comments igual
                attempt = 0
                while True:
                    try:
                        post.comments.replace_more(limit=0) # flatten tree
                        comments = post.comments.list()
                        break # salimos del loop en caso de éxito
                    
                    except TooManyRequests as e:
                        logging.warning(f"Rate limit hit when fetching comments for post {post.id}: {e}")
                        backoff_sleep(attempt)
                        attempt += 1
                    except RequestException as e:
                        logging.error(f"Request exception when fetching comments for post {post.id}: {e}")
                        backoff_sleep(attempt)
                        attempt += 1
                    except Exception as e:
                        logging.error(f"Unexpected error when fetching comments for post {post.id}: {e}")
                        comments = []
                        break  # Saltamos al siguiente post en caso de error
                    
                
                for comment in comments:
                    comment_labels = [] # creando la lista vacía de labels
                    if not isinstance(comment, praw.models.Comment):
                        continue  # Skippear si no es un comment válido

                    # añadiendo etiquetas la lista de comment_labels
                    if contains_trump_keyword(comment.body) and is_relevant_content(comment.body):
                        comment_labels.append('trump')
                    if contains_trumper_keyword(comment.body) and is_relevant_content(comment.body):
                        comment_labels.append('trumper')
                    if contains_kamala_keyword(comment.body) and is_relevant_content(comment.body):
                        comment_labels.append('kamala')
                    if contains_kamaler_keyword(comment.body) and is_relevant_content(comment.body):
                        comment_labels.append('kamaler')
                    if contains_democrat_keyword(comment.body) and is_relevant_content(comment.body):
                        comment_labels.append('democrat')
                    if contains_republican_keyword(comment.body) and is_relevant_content(comment.body):
                        comment_labels.append('republican')

                    if len(comment_labels) == 0: # also get comments with no labels
                        comment_labels.append('none')
                    
                    # metiendolo en los datos:
                    new_row = pd.DataFrame([{
                            'text': comment.body,
                            'submission_type': 'comment',
                            'subreddit': subreddit_name,
                            'labels': comment_labels,
                            'id': id
                        }])
                    df_buffer = pd.concat([df_buffer, new_row], ignore_index=True)
                    records_collected += 1
                    i +=1
                                        
                    # guardando el progreso periódicamente para liberar memoria
                    if records_collected >= save_threshold:
                        save_progress(df_buffer)
                        df_buffer = pd.DataFrame(columns=columns)
                        records_collected = 0
                    
        logging.info(f"Finished scraping r/{subreddit_name}. i = {i}")
    
    # guardando en caso de que se quede algo en el buffer
    if not df_buffer.empty:
        save_progress(df_buffer)
        logging.info(f"Finished scraping all subreddits. i = {i}")
        logging.info(f"Number of unique posts scraped: {n_unique_posts}")
        logging.info(f"Total number of id's: {len(id_list)}")
    

# llamando la función
all_hotscraped()

# idea de procesamiento: tomar como referencia un post, y sentiment analysis a comments en base al post.

In [57]:
import pandas as pd
df = pd.read_csv(r"C:\Users\34616\Documents\4GEEKS\datos_gordos\reddit\Scraping_results\all_hotscrape_v3.csv")
df.head(20)

,text,submission_type,subreddit,labels,id
0,Weekly Discussion Thread - week beginning Augu...,title,SocialDemocracy,['none'],1f19fuf
1,What do we think about the PSL (Party for Soci...,comment,SocialDemocracy,['none'],1f19fuf
2,[Could I ask you guys to vote for either Woodc...,comment,SocialDemocracy,['none'],1f19fuf
3,Given that they appear to defend China against...,comment,SocialDemocracy,"['trumper', 'republican']",1f19fuf
4,Rules clarification: Posts about the US electi...,title,SocialDemocracy,['none'],1eq921x
5,Thank you. The US elections are very important...,comment,SocialDemocracy,['none'],1eq921x
6,The political naivety among my progressive fri...,title,SocialDemocracy,['democrat'],1f3d499
7,Best take I've seen on this is that the majori...,comment,SocialDemocracy,"['kamala', 'kamaler']",1f3d499
8,Jill Stein did not bother registering in enoug...,comment,SocialDemocracy,['kamala'],1f3d499
9,Yeah completely agree and you can see the effe...,comment,SocialDemocracy,['republican'],1f3d499


In [43]:
print(df.iloc[16, 0])

I really do hate this situation. Not the post itself—I actually agree with it. However, I can’t help but feel that they would find another reason not to vote for the Democrats, even if the Democrats were supporting the Palestinian people. I do dislike Harris because she isn’t doing enough to stop the genocide, but at the same time, it’s hard to fight a genocide abroad when there’s a risk of one happening at home if Trump returns to power if what I read about project 2025 is true. I know I am naive, so I could be wrong, so I’m open to being corrected. Don’t yell at me if I am wrong instead explain why I am wrong please. Also sorry if my grammar or typing is bad, I’m not good at this.


In [58]:
print(df.shape)
print(f"Total duplicates: {df.duplicated().sum()}")
df = df.drop_duplicates()
print(f"Total NaNs: {df.isna().sum()}")
df = df.dropna()
print(df.shape)

(247582, 5)
Total duplicates: 7075
Total NaNs: text               3
submission_type    0
subreddit          0
labels             0
id                 0
dtype: int64
(240504, 5)


In [59]:
print(df.shape)

(240504, 5)


In [60]:
print(df.shape)
print(len(df['id'].unique()))
print(df['submission_type'].value_counts())


(240504, 5)
6245
submission_type
comment    226987
title       13517
Name: count, dtype: int64


In [31]:
titles = df[df['submission_type'] == 'title']

vcs = titles['id'].value_counts()

vcsrepes = vcs[vcs>1]
len(vcsrepes)



909

In [78]:
duplicate_ids = titles[titles.duplicated('id', keep=False)]['id'].unique()

print(len(duplicate_ids))

repes = titles[titles['id'].isin(duplicate_ids)]

repes = repes.sort_values(by='id')
repes


909


,text,submission_type,subreddit,labels,id
186591,I know they're just happy kids celebrating a w...,title,Liberal,['republicans'],1dy44ap
200642,I know they're just happy kids celebrating a w...,title,ModeratePolitics,['republicans'],1dy44ap
130387,I know they're just happy kids celebrating a w...,title,WayOfTheBern,['republicans'],1dy44ap
144438,I know they're just happy kids celebrating a w...,title,politics,['republicans'],1dy44ap
158489,I know they're just happy kids celebrating a w...,title,PoliticalDiscussion,['republicans'],1dy44ap
...,...,...,...,...,...
130715,Bangladesh: the ‘Global South’ debt crisis int...,title,politics,['none'],1f3j7qw
172868,Bangladesh: the ‘Global South’ debt crisis int...,title,Liberal,['none'],1f3j7qw
116664,Bangladesh: the ‘Global South’ debt crisis int...,title,WayOfTheBern,['none'],1f3j7qw
186919,Bangladesh: the ‘Global South’ debt crisis int...,title,ModeratePolitics,['none'],1f3j7qw


In [87]:
print(f"Text duplicates: {df['text'].duplicated().sum()}")
df['text'] = df['text'].drop_duplicates()
print(f"Nans (removed dupes):\n {df.isna().sum()}")
df = df.dropna()
df

Text duplicates: 0
Nans (removed dupes):
 text               0
submission_type    0
subreddit          0
labels             0
id                 0
dtype: int64


C:\Users\34616\AppData\Local\Temp\ipykernel_24276\469040774.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].drop_duplicates()


,text,submission_type,subreddit,labels,id
0,Weekly Discussion Thread - week beginning Augu...,title,SocialDemocracy,['none'],1f19fuf
1,What do we think about the PSL (Party for Soci...,comment,SocialDemocracy,['none'],1f19fuf
2,[Could I ask you guys to vote for either Woodc...,comment,SocialDemocracy,['none'],1f19fuf
3,Given that they appear to defend China against...,comment,SocialDemocracy,"['trumper', 'republican']",1f19fuf
4,Rules clarification: Posts about the US electi...,title,SocialDemocracy,['none'],1eq921x
...,...,...,...,...,...
247577,"In today's news, the water is wet",comment,Republican,['none'],1eg2q8p
247578,This is exactly why I plan to vote Trump,comment,Republican,['trump'],1eg2q8p
247579,"They rate it that way because it is, and Trump...",comment,Republican,['trump'],1eg2q8p
247580,Don't tell Lichtman,comment,Republican,['none'],1eg2q8p


In [88]:
print(df.shape)
print(len(df['id'].unique()))
print(df['submission_type'].value_counts())

(126260, 5)
6211
submission_type
comment    120193
title        6067
Name: count, dtype: int64


In [89]:
titles = df[df['submission_type'] == 'title']
duplicate_ids = titles[titles.duplicated('id', keep=False)]['id'].unique()
print(len(duplicate_ids))
repes = titles[titles['id'].isin(duplicate_ids)]
repes = repes.sort_values(by='id')
repes

0


,text,submission_type,subreddit,labels,id


In [90]:
df.to_csv(r"C:\Users\34616\Documents\4GEEKS\datos_gordos\reddit\Scraping_results\all_hotscrape_v3.csv")


In [91]:
df_check = pd.read_csv(r"C:\Users\34616\Documents\4GEEKS\datos_gordos\reddit\Scraping_results\all_hotscrape_v3.csv")
df_check.head()

,Unnamed: 0,text,submission_type,subreddit,labels,id
0,0,Weekly Discussion Thread - week beginning Augu...,title,SocialDemocracy,['none'],1f19fuf
1,1,What do we think about the PSL (Party for Soci...,comment,SocialDemocracy,['none'],1f19fuf
2,2,[Could I ask you guys to vote for either Woodc...,comment,SocialDemocracy,['none'],1f19fuf
3,3,Given that they appear to defend China against...,comment,SocialDemocracy,"['trumper', 'republican']",1f19fuf
4,4,Rules clarification: Posts about the US electi...,title,SocialDemocracy,['none'],1eq921x


In [93]:
print(df.shape)
print(df.isna().sum())
print(df.duplicated().sum())
print("text: ")
print(df['text'].duplicated().sum())
print(df['text'].isna().sum())


(126260, 5)
text               0
submission_type    0
subreddit          0
labels             0
id                 0
dtype: int64
0
text: 
0
0


In [96]:
subreddits_original_list = ['SocialDemocracy', 'USPolitics', 'Libertarian', 'ConservativesOnly', 'OurPresident', 'Socialism', 'WayOfTheBern', 'politics', 'PoliticalDiscussion', 'Conservative', 'Liberal', 'ModeratePolitics', 'Ask_Politics', 'Democrats', 'Republican']
subreddits_original_list

['SocialDemocracy',
 'USPolitics',
 'Libertarian',
 'ConservativesOnly',
 'OurPresident',
 'Socialism',
 'WayOfTheBern',
 'politics',
 'PoliticalDiscussion',
 'Conservative',
 'Liberal',
 'ModeratePolitics',
 'Ask_Politics',
 'Democrats',
 'Republican']

In [95]:
df_check['subreddit'].value_counts()

subreddit
Libertarian          34060
OurPresident         31177
SocialDemocracy      25786
Republican           16843
Socialism            12394
USPolitics            4360
ConservativesOnly     1640
Name: count, dtype: int64